In [1]:
# IBL SWR detector
import os
import re
import subprocess
import numpy as np
import pandas as pd
from scipy import io, signal, stats
from scipy.signal import lfilter
import scipy.ndimage
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_filter1d
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import ripple_detection
from ripple_detection import filter_ripple_band
import ripple_detection.simulate as ripsim  # for making our time vectors
from tqdm import tqdm
from iblatlas.atlas import AllenAtlas
from iblatlas.regions import BrainRegions
import spikeglx
from brainbox.io.one import load_channel_locations
from brainbox.io.spikeglx import Streamer
from brainbox.io.one import SpikeSortingLoader
from neurodsp.voltage import destripe_lfp
from ibllib.plots import Density
import time
import traceback
import logging
import logging.handlers
import sys
from fitter import Fitter, get_common_distributions, get_distributions
import pickle
from one.api import ONE


ONE.setup(base_url="https://openalyx.internationalbrainlab.org", silent=True)
one = ONE(password="international")

/home/acampbell/miniconda3/envs/ONE_ibl_env/lib/python3.10/site-packages/neurodsp/__init__.py:5: FutureWarning: neurodsp has been renamed to ibldsp and the old name will be deprecated on 01-Sep-2024.
  warn(


## Making Brain Images of Probes

Using this code you can take snapshots of the probe locations in 

https://community.brain-map.org/t/how-to-switch-between-the-3-coordinate-systems-of-mouse-connectivity/952 <br>
https://community.brain-map.org/t/how-to-transform-ccf-x-y-z-coordinates-into-stereotactic-coordinates/1858

In [111]:
seshes = os.listdir(
    "/space/scratch/allen_visbehave_swr_data/allen_visbehave_swr_murphylab2024"
)
seshes = [s.replace("swrs_session_", "") for s in seshes if "swrs_session_" in s]
seshes = [int(s) for s in seshes]

In [112]:
import random
import os
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import vedo
from brainrender import Scene
from brainrender.actors import Points
from vedo import Plotter  # <- this will be used to render an embedded scene

# from vedo import embedWindow
vedo.settings.default_backend = "k3d"

from iblatlas.atlas import AllenAtlas
from iblatlas.regions import BrainRegions

ba = AllenAtlas()
br = BrainRegions()  # br is also an attribute of ba so could to br = ba.regions

In [113]:
def coords_transform(ml, ap, dv, resolution=25):

    ml_ccf = ml + 5700
    # ap_ccf = ((-1)*ap + 5400)/resolution
    ap_ccf = -ap + 4700
    dv_ccf = -dv

    return ml_ccf, ap_ccf, dv_ccf

In [114]:
# Display the Allen Brain mouse atlas.
scene = Scene(atlas_name="allen_mouse_25um", title="Cells in primary visual cortex")
# Make sure it gets embedded in the window
scene.jupyter = True


def get_n_random_points_in_region(region, N):
    """
    Gets N random points inside (or on the surface) of a mes
    """

    region_bounds = region.mesh.bounds()
    X = np.random.randint(region_bounds[0], region_bounds[1], size=10000)
    Y = np.random.randint(region_bounds[2], region_bounds[3], size=10000)
    Z = np.random.randint(region_bounds[4], region_bounds[5], size=10000)
    pts = [[x, y, z] for x, y, z in zip(X, Y, Z)]

    ipts = region.mesh.inside_points(pts).coordinates
    return np.vstack(random.choices(ipts, k=N))


scene = Scene(title="Labelled cells")

# Display a brain region
cornu_Ammonis_1 = scene.add_brain_region("CA1", alpha=0.2)

ca1_coordinates = []

# Iterate over all subdirectories in the base directory
for dir_name in os.listdir(ibl_ripples_path):
    if "swrs_session_" in dir_name:
        # Extract the session ID from the directory name and get the session data
        session_id = dir_name.split("swrs_session_")[1]
        print(session_id)
        probelist, probenames = one.eid2pid(
            session_id
        )  # probe_id is pid in the IBL tutorials

        # Iterate over all files in the directory
        for file_name in os.listdir(os.path.join(ibl_ripples_path, dir_name)):
            print(file_name)
            ca1_coordinates = []
            if "karlsson" in file_name:
                # Extract probe_id from the filename
                probe_id = re.search("probe_(.*?)_", file_name).group(1)
                print(probe_id)
                p_idx = np.array([p == probe_id for p in probelist])
                probe_name = np.array(probenames)[p_idx]
                probe_name = probe_name[0]
                collectionname = f"alf/{probe_name}/pykilosort"  # ensures channels are all from this probe
                channels = one.load_object(
                    session_id, "channels", collection=collectionname
                )
                channels.allen2017_25um_acronym = br.id2acronym(
                    channels["brainLocationIds_ccf_2017"]
                )
                ca1index = channels["allen2017_25um_acronym"] == "CA1"
                # Get the coordinates for all CA1 channels specific to the probe_id
                ca1_coordinates = channels["mlapdv"][ca1index]
                transformed_coordinates = np.array(
                    [
                        coords_transform(ml, ap, dv, resolution=25)
                        for ml, ap, dv in ca1_coordinates
                    ]
                )

                # rearanging it so that it can be plotted
                transformed_coordinates = np.take(
                    transformed_coordinates, [1, 2, 0], axis=1
                )

                # Extract the dorsal_ventral_ccf_coordinate values
                dorsal_ventral_values = transformed_coordinates[:, 1]

                # Sort the values
                sorted_values = np.sort(dorsal_ventral_values)

                # Calculate the mid-point
                mid_point_value = sorted_values[len(sorted_values) // 2]

                # Find the row with the mid-point value
                mid_point_row = transformed_coordinates[
                    np.where(transformed_coordinates[:, 1] == mid_point_value)
                ]

                scene.add(
                    Points(
                        mid_point_row,
                        name="probe_1",
                        colors="black",
                        radius=100,
                    )
                )

# Add label to the brain region
scene.add_label(cornu_Ammonis_1, "CA1")

# render
scene.content
scene.render()

plt = Plotter()
plt.show(*scene.renderables)

0cc486c3-8c7b-494d-aa04-b70e2690bcba
probe_db2100c3-64ac-463e-97a1-20ce8266cd5f_channel_channelsrawInd_366_karlsson_detector_events.csv
db2100c3-64ac-463e-97a1-20ce8266cd5f
session_0cc486c3-8c7b-494d-aa04-b70e2690bcba_putative_global_swrs_no_movement.csv
probe_db2100c3-64ac-463e-97a1-20ce8266cd5f_channel_channelsrawInd_138_movement_artifacts.csv
probe_db2100c3-64ac-463e-97a1-20ce8266cd5f_channel_channelsrawInd_319_movement_artifacts.csv
session_0cc486c3-8c7b-494d-aa04-b70e2690bcba_putative_global_swrs_no_movement_no_gamma.csv
probe_db2100c3-64ac-463e-97a1-20ce8266cd5f_channel_channelsrawInd_366_gamma_band_events.csv
746d1902-fa59-4cab-b0aa-013be36060d5
probe_e31b4e39-e350-47a9-aca4-72496d99ff2a_channel_channelsrawInd_351_movement_artifacts.csv
session_746d1902-fa59-4cab-b0aa-013be36060d5_putative_global_swrs_no_movement.csv
session_746d1902-fa59-4cab-b0aa-013be36060d5_putative_global_swrs_no_movement_no_gamma.csv
probe_e31b4e39-e350-47a9-aca4-72496d99ff2a_channel_channelsrawInd_383_gam

probe_9e44ddb5-7c7c-48f1-954a-6cec2ad26088_channel_channelsrawInd_234_gamma_band_events.csv
c7bf2d49-4937-4597-b307-9f39cb1c7b16
probe_e49f221d-399d-4297-bb7d-2d23cc0e4acc_channel_channelsrawInd_225_karlsson_detector_events.csv
e49f221d-399d-4297-bb7d-2d23cc0e4acc


/home/acampbell/Downloads/ONE/openalyx.internationalbrainlab.org/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy: 100%|██████████| 3.20k/3.20k [00:00<00:00, 14.0kB/s]
/home/acampbell/Downloads/ONE/openalyx.internationalbrainlab.org/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy: 100%|██████████| 4.74k/4.74k [00:00<00:00, 24.0kB/s]


probe_e49f221d-399d-4297-bb7d-2d23cc0e4acc_channel_channelsrawInd_166_movement_artifacts.csv
session_c7bf2d49-4937-4597-b307-9f39cb1c7b16_putative_global_swrs_no_movement.csv
probe_e49f221d-399d-4297-bb7d-2d23cc0e4acc_channel_channelsrawInd_225_gamma_band_events.csv
probe_e49f221d-399d-4297-bb7d-2d23cc0e4acc_channel_channelsrawInd_131_movement_artifacts.csv
session_c7bf2d49-4937-4597-b307-9f39cb1c7b16_putative_global_swrs_no_movement_no_gamma.csv
b03fbc44-3d8e-4a6c-8a50-5ea3498568e0
session_b03fbc44-3d8e-4a6c-8a50-5ea3498568e0_putative_global_swrs_no_movement_no_gamma.csv
probe_9117969a-3f0d-478b-ad75-98263e3bfacf_channel_channelsrawInd_257_karlsson_detector_events.csv
9117969a-3f0d-478b-ad75-98263e3bfacf
session_b03fbc44-3d8e-4a6c-8a50-5ea3498568e0_putative_global_swrs_no_movement.csv
probe_9117969a-3f0d-478b-ad75-98263e3bfacf_channel_channelsrawInd_306_movement_artifacts.csv
probe_9117969a-3f0d-478b-ad75-98263e3bfacf_channel_channelsrawInd_257_gamma_band_events.csv
probe_9117969a-3f0

Cannot run function add_label in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

╭─────────────────────────────────╮
│                                 │
│   Scene actors                  │
│   ────────────                  │
│                                 │
│   - root (type: brain region)   │
│   - title (type: title)         │
│   - CA1 (type: brain region)    │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: Points)      │
│   - probe_1 (type: [38;2

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…